# Introduction to Policy Gradients

Authors: Sean Lin, Jaiveer Singh, Ethan Mehta

Estimated Time: 1 hr. 30 min.

References: OpenAI Spinning Up (https://spinningup.openai.com/en/latest/index.html)
<br> OpenAI's open source resource on reinforcement learning, Spinning Up, provides utils files and skeleton code for vanilla policy gradients that we built upon for this assignment

## Introduction

## Setup

In [1]:
import numpy as np
import tensorflow as tf
import gym
import time
import core
from utils.logx import EpochLogger
from utils.mpi_tf import MpiAdamOptimizer, sync_all_params
from utils.mpi_tools import mpi_fork, mpi_avg, proc_id, mpi_statistics_scalar, num_procs
from utils.run_utils import setup_logger_kwargs

## Policies in continuous state space, discrete action space

Recall that a policy is a rule used to decide what actions to take given the state or observations of the world. We can conceptualize a policy as a mapping from input observations to an output action. In the previous problem, we looked at worlds with discrete state spaces and discrete action spaces; for instance in the taxi problem, there are a discrete set of states that the taxi and passenger can be in. Likewise, there was a discrete set of actions that the taxi could take (move up, move down, move left, move right, pick up, and drop off). The policy takes the form of a dictionary, where one state maps to one action.

The self-balancing segway presents a different challenge. In this world, the segway can move anywhere on the line, and lean in any way. The observations that we gather take on the form:
$$[\text{position of cart}, \text{velocity of cart}, \text{angle of pole}, \text{rotation rate of pole}]$$

In other words, the state space is continuous. We now see that our previous notion of a policy no longer applies––we cannot represent a policy as a dictionary because the set of states, which served as the keys to the dictionary, are infinite! Therefore, we must formulate our policy in some other way.

This is where neural networks come in handy. Given continuous observations as input, we can develop a classification neural network that 

### Task: Implement Categorical Policy

In this task, we will define a categorical policy computed via a multilayer perceptron. Fill in the function `mlp`

In [2]:
EPS = 1e-8

def mlp(x, hidden_sizes=[64,64,2], activation=tf.tanh, output_activation=None):
    for h in hidden_sizes[:-1]:
        x = tf.layers.dense(x, units=h, activation=activation)
    return tf.layers.dense(x, units=hidden_sizes[-1], activation=output_activation)

def mlp_categorical_policy(x, a, hidden_sizes, activation, output_activation, action_space):
    act_dim = action_space.n
    logits = mlp(x, list(hidden_sizes)+[act_dim], activation, None)
    logp_all = tf.nn.log_softmax(logits)
    pi = tf.squeeze(tf.multinomial(logits,1), axis=1)
    logp = tf.reduce_sum(tf.one_hot(a, depth=act_dim) * logp_all, axis=1)
    logp_pi = tf.reduce_sum(tf.one_hot(pi, depth=act_dim) * logp_all, axis=1)
    return pi, logp, logp_pi

## Vanilla Policy Gradients

In [3]:
class VPGBuffer:
    """
    A buffer for storing trajectories experienced by a VPG agent interacting
    with the environment, and using Generalized Advantage Estimation (GAE-Lambda)
    for calculating the advantages of state-action pairs.
    """

    def __init__(self, obs_dim, act_dim, size, gamma=0.99, lam=0.95):
        self.obs_buf = np.zeros(core.combined_shape(size, obs_dim), dtype=np.float32)
        self.act_buf = np.zeros(core.combined_shape(size, act_dim), dtype=np.float32)
        self.adv_buf = np.zeros(size, dtype=np.float32)
        self.rew_buf = np.zeros(size, dtype=np.float32)
        self.ret_buf = np.zeros(size, dtype=np.float32)
        self.val_buf = np.zeros(size, dtype=np.float32)
        self.logp_buf = np.zeros(size, dtype=np.float32)
        self.gamma, self.lam = gamma, lam
        self.ptr, self.path_start_idx, self.max_size = 0, 0, size

    def store(self, obs, act, rew, val, logp):
        """
        Append one timestep of agent-environment interaction to the buffer.
        """
        assert self.ptr < self.max_size     # buffer has to have room so you can store
        self.obs_buf[self.ptr] = obs
        self.act_buf[self.ptr] = act
        self.rew_buf[self.ptr] = rew
        self.val_buf[self.ptr] = val
        self.logp_buf[self.ptr] = logp
        self.ptr += 1

    def finish_path(self, last_val=0):
        """
        Call this at the end of a trajectory, or when one gets cut off
        by an epoch ending. This looks back in the buffer to where the
        trajectory started, and uses rewards and value estimates from
        the whole trajectory to compute advantage estimates with GAE-Lambda,
        as well as compute the rewards-to-go for each state, to use as
        the targets for the value function.

        The "last_val" argument should be 0 if the trajectory ended
        because the agent reached a terminal state (died), and otherwise
        should be V(s_T), the value function estimated for the last state.
        This allows us to bootstrap the reward-to-go calculation to account
        for timesteps beyond the arbitrary episode horizon (or epoch cutoff).
        """

        path_slice = slice(self.path_start_idx, self.ptr)
        rews = np.append(self.rew_buf[path_slice], last_val)
        vals = np.append(self.val_buf[path_slice], last_val)
        
        # the next two lines implement GAE-Lambda advantage calculation
        deltas = rews[:-1] + self.gamma * vals[1:] - vals[:-1]
        self.adv_buf[path_slice] = core.discount_cumsum(deltas, self.gamma * self.lam)
        
        # the next line computes rewards-to-go, to be targets for the value function
        self.ret_buf[path_slice] = core.discount_cumsum(rews, self.gamma)[:-1]
        
        self.path_start_idx = self.ptr

    def get(self):
        """
        Call this at the end of an epoch to get all of the data from
        the buffer, with advantages appropriately normalized (shifted to have
        mean zero and std one). Also, resets some pointers in the buffer.
        """
        assert self.ptr == self.max_size    # buffer has to be full before you can get
        self.ptr, self.path_start_idx = 0, 0
        # the next two lines implement the advantage normalization trick
        adv_mean, adv_std = mpi_statistics_scalar(self.adv_buf)
        self.adv_buf = (self.adv_buf - adv_mean) / adv_std
        return [self.obs_buf, self.act_buf, self.adv_buf, 
                self.ret_buf, self.logp_buf]



def vpg(env_fn, actor_critic=core.mlp_actor_critic, ac_kwargs=dict(), seed=0, 
        steps_per_epoch=4000, epochs=50, gamma=0.99, pi_lr=3e-4,
        vf_lr=1e-3, train_v_iters=80, lam=0.97, max_ep_len=1000,
        logger_kwargs=dict(), save_freq=10):
    """
    Vanilla Policy Gradient 

    (with GAE-Lambda for advantage estimation)

    Args:
        env_fn : A function which creates a copy of the environment.
            The environment must satisfy the OpenAI Gym API.

        actor_critic: A function which takes in placeholder symbols 
            for state, ``x_ph``, and action, ``a_ph``, and returns the main 
            outputs from the agent's Tensorflow computation graph:

            ===========  ================  ======================================
            Symbol       Shape             Description
            ===========  ================  ======================================
            ``pi``       (batch, act_dim)  | Samples actions from policy given 
                                           | states.
            ``logp``     (batch,)          | Gives log probability, according to
                                           | the policy, of taking actions ``a_ph``
                                           | in states ``x_ph``.
            ``logp_pi``  (batch,)          | Gives log probability, according to
                                           | the policy, of the action sampled by
                                           | ``pi``.
            ``v``        (batch,)          | Gives the value estimate for states
                                           | in ``x_ph``. (Critical: make sure 
                                           | to flatten this!)
            ===========  ================  ======================================

        ac_kwargs (dict): Any kwargs appropriate for the actor_critic 
            function you provided to VPG.

        seed (int): Seed for random number generators.

        steps_per_epoch (int): Number of steps of interaction (state-action pairs) 
            for the agent and the environment in each epoch.

        epochs (int): Number of epochs of interaction (equivalent to
            number of policy updates) to perform.

        gamma (float): Discount factor. (Always between 0 and 1.)

        pi_lr (float): Learning rate for policy optimizer.

        vf_lr (float): Learning rate for value function optimizer.

        train_v_iters (int): Number of gradient descent steps to take on 
            value function per epoch.

        lam (float): Lambda for GAE-Lambda. (Always between 0 and 1,
            close to 1.)

        max_ep_len (int): Maximum length of trajectory / episode / rollout.

        logger_kwargs (dict): Keyword args for EpochLogger.

        save_freq (int): How often (in terms of gap between epochs) to save
            the current policy and value function.

    """

    logger = EpochLogger(**logger_kwargs)
    logger.save_config(locals())

    seed += 10000 * proc_id()
    tf.set_random_seed(seed)
    np.random.seed(seed)

    env = env_fn()
    obs_dim = env.observation_space.shape
    act_dim = env.action_space.shape
    
    # Share information about action space with policy architecture
    ac_kwargs['action_space'] = env.action_space

    # Inputs to computation graph
    x_ph, a_ph = core.placeholders_from_spaces(env.observation_space, env.action_space)
    adv_ph, ret_ph, logp_old_ph = core.placeholders(None, None, None)

    # Main outputs from computation graph
    pi, logp, logp_pi, v = actor_critic(x_ph, a_ph, **ac_kwargs)

    # Need all placeholders in *this* order later (to zip with data from buffer)
    all_phs = [x_ph, a_ph, adv_ph, ret_ph, logp_old_ph]

    # Every step, get: action, value, and logprob
    get_action_ops = [pi, v, logp_pi]

    # Experience buffer
    local_steps_per_epoch = int(steps_per_epoch / num_procs())
    buf = VPGBuffer(obs_dim, act_dim, local_steps_per_epoch, gamma, lam)

    # Count variables
    var_counts = tuple(core.count_vars(scope) for scope in ['pi', 'v'])
    logger.log('\nNumber of parameters: \t pi: %d, \t v: %d\n'%var_counts)

    # VPG objectives
    pi_loss = -tf.reduce_mean(logp * adv_ph)
    v_loss = tf.reduce_mean((ret_ph - v)**2)

    # Info (useful to watch during learning)
    approx_kl = tf.reduce_mean(logp_old_ph - logp)      # a sample estimate for KL-divergence, easy to compute
    approx_ent = tf.reduce_mean(-logp)                  # a sample estimate for entropy, also easy to compute

    # Optimizers
    train_pi = MpiAdamOptimizer(learning_rate=pi_lr).minimize(pi_loss)
    train_v = MpiAdamOptimizer(learning_rate=vf_lr).minimize(v_loss)

    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    # Sync params across processes
    sess.run(sync_all_params())

    # Setup model saving
    logger.setup_tf_saver(sess, inputs={'x': x_ph}, outputs={'pi': pi, 'v': v})

    def update():
        inputs = {k:v for k,v in zip(all_phs, buf.get())}
        pi_l_old, v_l_old, ent = sess.run([pi_loss, v_loss, approx_ent], feed_dict=inputs)

        # Policy gradient step
        sess.run(train_pi, feed_dict=inputs)

        # Value function learning
        for _ in range(train_v_iters):
            sess.run(train_v, feed_dict=inputs)

        # Log changes from update
        pi_l_new, v_l_new, kl = sess.run([pi_loss, v_loss, approx_kl], feed_dict=inputs)
        logger.store(LossPi=pi_l_old, LossV=v_l_old, 
                     KL=kl, Entropy=ent, 
                     DeltaLossPi=(pi_l_new - pi_l_old),
                     DeltaLossV=(v_l_new - v_l_old))

    start_time = time.time()
    o, ep_ret, ep_len = env.reset(), 0, 0

    # Main loop: collect experience in env and update/log each epoch
    for epoch in range(epochs):
        for t in range(local_steps_per_epoch):
            a, v_t, logp_t = sess.run(get_action_ops, feed_dict={x_ph: o.reshape(1,-1)})

            o2, r, d, _ = env.step(a[0])
            env.render()
            ep_ret += r
            ep_len += 1

            # save and log
            buf.store(o, a, r, v_t, logp_t)
            logger.store(VVals=v_t)

            # Update obs (critical!)
            o = o2

            terminal = d or (ep_len == max_ep_len)
            if terminal or (t==local_steps_per_epoch-1):
                if not(terminal):
                    print('Warning: trajectory cut off by epoch at %d steps.'%ep_len)
                # if trajectory didn't reach terminal state, bootstrap value target
                last_val = 0 if d else sess.run(v, feed_dict={x_ph: o.reshape(1,-1)})
                buf.finish_path(last_val)
                if terminal:
                    # only save EpRet / EpLen if trajectory finished
                    logger.store(EpRet=ep_ret, EpLen=ep_len)
                o, ep_ret, ep_len = env.reset(), 0, 0

        # Save model
        if (epoch % save_freq == 0) or (epoch == epochs-1):
            logger.save_state({'env': env}, None)

        # Perform VPG update!
        update()

        # Log info about epoch
        logger.log_tabular('Epoch', epoch)
        logger.log_tabular('EpRet', with_min_and_max=True)
        logger.log_tabular('EpLen', average_only=True)
        logger.log_tabular('VVals', with_min_and_max=True)
        logger.log_tabular('TotalEnvInteracts', (epoch+1)*steps_per_epoch)
        logger.log_tabular('LossPi', average_only=True)
        logger.log_tabular('LossV', average_only=True)
        logger.log_tabular('DeltaLossPi', average_only=True)
        logger.log_tabular('DeltaLossV', average_only=True)
        logger.log_tabular('Entropy', average_only=True)
        logger.log_tabular('KL', average_only=True)
        logger.log_tabular('Time', time.time()-start_time)
        logger.dump_tabular()

In [5]:
env = 'CartPole-v1'

### YOUR CODE HERE ###
# hidden_layers = 
# depth = 
# steps = 
# epochs = 

# ANSWER 
hidden_layers = 64
depth = 2
steps = 4000
epochs = 100

### END ###
gamma = 0.99
seed = 0
# parser.add_argument('--cpu', type=int, default=2)
exp_name = 'vpg'

# Reset the default graph to prevent errors on multiple runs of Vanilla Policy Gradient
tf.reset_default_graph()
logger_kwargs = setup_logger_kwargs(exp_name, seed)

vpg(lambda : gym.make(env), actor_critic=core.mlp_actor_critic,
    ac_kwargs=dict(policy=mlp_categorical_policy, hidden_sizes=[hidden_layers]*depth), gamma=gamma, 
    seed=seed, steps_per_epoch=steps, epochs=epochs,
    logger_kwargs=logger_kwargs)

Logging data to /Users/seanlin/Desktop/MehtaKnights-189/vpg-problem/data/vpg/vpg_s0/progress.txt
Saving config:

{
    "ac_kwargs":	{
        "hidden_sizes":	[
            64,
            64
        ],
        "policy":	"mlp_categorical_policy"
    },
    "actor_critic":	"mlp_actor_critic",
    "env_fn":	"<function <lambda> at 0x1385cb8c8>",
    "epochs":	100,
    "exp_name":	"vpg",
    "gamma":	0.99,
    "lam":	0.97,
    "logger":	{
        "<utils.logx.EpochLogger object at 0x137493780>":	{
            "epoch_dict":	{},
            "exp_name":	"vpg",
            "first_row":	true,
            "log_current_row":	{},
            "log_headers":	[],
            "output_dir":	"/Users/seanlin/Desktop/MehtaKnights-189/vpg-problem/data/vpg/vpg_s0",
            "output_file":	{
                "<_io.TextIOWrapper name='/Users/seanlin/Desktop/MehtaKnights-189/vpg-problem/data/vpg/vpg_s0/progress.txt' mode='w' encoding='UTF-8'>":	{
                    "mode":	"w"
                }
            }

/Users/seanlin/opt/anaconda3/envs/spinningup/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))




Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use `tf.random.categorical` instead.


Number of parameters: 	 pi: 4610, 	 v: 4545

Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule a

---------------------------------------
|             Epoch |               5 |
|      AverageEpRet |            31.7 |
|          StdEpRet |            16.6 |
|          MaxEpRet |              81 |
|          MinEpRet |              10 |
|             EpLen |            31.7 |
|      AverageVVals |            18.3 |
|          StdVVals |            4.84 |
|          MaxVVals |            23.2 |
|          MinVVals |           -5.67 |
| TotalEnvInteracts |         2.4e+04 |
|            LossPi |         -0.0318 |
|             LossV |             121 |
|       DeltaLossPi |        -0.00305 |
|        DeltaLossV |           -7.92 |
|           Entropy |           0.679 |
|                KL |        0.000365 |
|              Time |             307 |
---------------------------------------
---------------------------------------
|             Epoch |               6 |
|      AverageEpRet |            32.3 |
|          StdEpRet |            21.1 |
|          MaxEpRet |             127 |


---------------------------------------
|             Epoch |              15 |
|      AverageEpRet |            50.5 |
|          StdEpRet |            31.7 |
|          MaxEpRet |             177 |
|          MinEpRet |              13 |
|             EpLen |            50.5 |
|      AverageVVals |            25.6 |
|          StdVVals |            9.85 |
|          MaxVVals |            35.4 |
|          MinVVals |           -1.82 |
| TotalEnvInteracts |         6.4e+04 |
|            LossPi |          -0.048 |
|             LossV |             281 |
|       DeltaLossPi |        -0.00201 |
|        DeltaLossV |           -20.9 |
|           Entropy |           0.655 |
|                KL |       -0.000146 |
|              Time |             455 |
---------------------------------------
---------------------------------------
|             Epoch |              16 |
|      AverageEpRet |            42.9 |
|          StdEpRet |            29.1 |
|          MaxEpRet |             133 |


...

---------------------------------------
|             Epoch |              55 |
|      AverageEpRet |             123 |
|          StdEpRet |            93.7 |
|          MaxEpRet |             401 |
|          MinEpRet |              22 |
|             EpLen |             123 |
|      AverageVVals |            50.4 |
|          StdVVals |            14.7 |
|          MaxVVals |            67.2 |
|          MinVVals |           -6.57 |
| TotalEnvInteracts |        2.24e+05 |
|            LossPi |         -0.0556 |
|             LossV |             517 |
|       DeltaLossPi |        -0.00101 |
|        DeltaLossV |           -79.9 |
|           Entropy |           0.614 |
|                KL |        0.000132 |
|              Time |        1.23e+03 |
---------------------------------------
---------------------------------------
|             Epoch |              56 |
|      AverageEpRet |             113 |
|          StdEpRet |            66.2 |
|          MaxEpRet |             336 |


---------------------------------------
|             Epoch |              65 |
|      AverageEpRet |             113 |
|          StdEpRet |            55.7 |
|          MaxEpRet |             238 |
|          MinEpRet |              23 |
|             EpLen |             113 |
|      AverageVVals |            52.1 |
|          StdVVals |            16.8 |
|          MaxVVals |            72.7 |
|          MinVVals |             7.6 |
| TotalEnvInteracts |        2.64e+05 |
|            LossPi |         -0.0663 |
|             LossV |             364 |
|       DeltaLossPi |        -0.00093 |
|        DeltaLossV |           -78.2 |
|           Entropy |           0.602 |
|                KL |        0.000171 |
|              Time |        1.36e+03 |
---------------------------------------
---------------------------------------
|             Epoch |              66 |
|      AverageEpRet |             143 |
|          StdEpRet |            74.2 |
|          MaxEpRet |             289 |


---------------------------------------
|             Epoch |              75 |
|      AverageEpRet |             154 |
|          StdEpRet |            50.6 |
|          MaxEpRet |             285 |
|          MinEpRet |              66 |
|             EpLen |             154 |
|      AverageVVals |            53.9 |
|          StdVVals |            21.3 |
|          MaxVVals |            78.8 |
|          MinVVals |            5.51 |
| TotalEnvInteracts |        3.04e+05 |
|            LossPi |         -0.0516 |
|             LossV |             280 |
|       DeltaLossPi |       -0.000639 |
|        DeltaLossV |           -56.6 |
|           Entropy |           0.593 |
|                KL |        3.07e-05 |
|              Time |        1.55e+03 |
---------------------------------------
---------------------------------------
|             Epoch |              76 |
|      AverageEpRet |             156 |
|          StdEpRet |            80.7 |
|          MaxEpRet |             305 |


---------------------------------------
|             Epoch |              85 |
|      AverageEpRet |             180 |
|          StdEpRet |            85.6 |
|          MaxEpRet |             425 |
|          MinEpRet |              40 |
|             EpLen |             180 |
|      AverageVVals |            52.5 |
|          StdVVals |            21.7 |
|          MaxVVals |            83.5 |
|          MinVVals |            5.97 |
| TotalEnvInteracts |        3.44e+05 |
|            LossPi |         -0.0403 |
|             LossV |             398 |
|       DeltaLossPi |       -0.000504 |
|        DeltaLossV |            -124 |
|           Entropy |           0.581 |
|                KL |        1.55e-05 |
|              Time |        1.69e+03 |
---------------------------------------
---------------------------------------
|             Epoch |              86 |
|      AverageEpRet |             197 |
|          StdEpRet |            82.4 |
|          MaxEpRet |             399 |


---------------------------------------
|             Epoch |              95 |
|      AverageEpRet |             263 |
|          StdEpRet |             135 |
|          MaxEpRet |             500 |
|          MinEpRet |              73 |
|             EpLen |             263 |
|      AverageVVals |            60.2 |
|          StdVVals |            17.7 |
|          MaxVVals |            79.2 |
|          MinVVals |            8.26 |
| TotalEnvInteracts |        3.84e+05 |
|            LossPi |         -0.0451 |
|             LossV |             416 |
|       DeltaLossPi |       -0.000437 |
|        DeltaLossV |            -184 |
|           Entropy |           0.577 |
|                KL |       -1.13e-05 |
|              Time |        1.96e+03 |
---------------------------------------
---------------------------------------
|             Epoch |              96 |
|      AverageEpRet |             188 |
|          StdEpRet |              56 |
|          MaxEpRet |             298 |
